# Mixer correction

Setup
-----

First, we are going to import the required packages.

In [192]:
# Import ipython widgets
import json
import math
import os

import ipywidgets as widgets
import matplotlib.pyplot
import numpy as np
import rich
import pandas as pd

# Set up the environment.
import scipy.signal
from IPython.display import display
from ipywidgets import fixed, interact, interact_manual, interactive

from qblox_instruments import Cluster, PlugAndPlay, Pulsar
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [193]:
# This is the table that we want to populate
calibration = pd.DataFrame()
calibration["module"]  = []
calibration["complex_output"] = []
calibration["sequencer index"] = []
calibration["lo_freq (Hz)"] = []
calibration["if (Hz)"] = []
calibration["dc_mixer_offset_I"] = []
calibration["dc_mixer_offset_Q"] = []
calibration["mixer_amp_ratio"] = []
calibration["mixer_phase_error_deg"] = []
calibration

,module,complex_output,sequencer index,lo_freq (Hz),if (Hz),dc_mixer_offset_I,dc_mixer_offset_Q,mixer_amp_ratio,mixer_phase_error_deg


## Create Q1ASM program to play on the device

In [194]:
#Waveform dictionary (data will hold the samples and index will be used to select the waveforms in the instrument).
waveforms = {
             "I":     {"data": [], "index": 0},
             "Q":     {"data": [], "index": 1}
            }

waveforms["I"]["data"] = [0.4]*8000
waveforms["Q"]["data"] = [0.0]*8000

#Sequence program.
prog = f"start: set_mrk 15 \n play 0,1,1000\n set_mrk 15 \n play 0,1,1000\njmp @start"

#Sequence program which stops.
# prog = """
#         move 1250000, R0
#         set_mrk    15
# start:
#         reset_ph
#         upd_param  4
#         play       0,1,4
#         wait       7996
#         loop       R0,@start
#         set_mrk    0
#         upd_param  4
#         stop
# """

#Reformat waveforms to lists if necessary.
for name in waveforms:
    if str(type(waveforms[name]["data"]).__name__) == "ndarray":
        waveforms[name]["data"] = waveforms[name]["data"].tolist()  # JSON only supports lists

#Add sequence program and waveforms to single dictionary and write to JSON file.
wave_and_prog_dict = {"waveforms": waveforms, "weights": {}, "acquisitions": {}, "program": prog}
with open("sequence.json", 'w', encoding='utf-8') as file:
    json.dump(wave_and_prog_dict, file, indent=4)
    file.close()

## Select Device

We scan for the available devices connected via ethernet using the Plug & Play functionality of the Qblox Instruments package (see [Plug & Play](https://qblox-qblox-instruments.readthedocs-hosted.com/en/master/api_reference/pnp.html) for more info).

In [195]:
# Scan for available devices and display
with PlugAndPlay() as p:
    # get info of all devices
    device_list = p.list_devices()
    device_keys = list(device_list.keys())

# rich.print(device_list)
# create widget for names and ip addresses
connect = widgets.Dropdown(
    options=[(device_list[key]["identity"]["ip"]) for key in device_list.keys()],
    description="Select Device",
)
display(connect)

Dropdown(description='Select Device', options=('192.0.2.143', '192.0.2.141', '192.0.2.142', '192.0.2.72'), val…

In [221]:
# close all previous connections to the cluster
Cluster.close_all()

#device_name = "pingu_cluster"
device_name = "clusterA"
# ip_address = connect.v./alue
ip_address = '192.0.2.141'
# connect to the cluster and reset
cluster = Cluster(device_name, ip_address)
cluster.reset()
print(f"{device_name} connected at {ip_address}")
# cluster.identify()

clusterA connected at 192.0.2.141


## Select Module in Device

In [222]:
# Find all QRM/QCM modules
cluster.reset()
available_slots = {}
for module in cluster.modules:
    # if module is currently present in stack
    if cluster._get_modules_present(module.slot_idx):
        # check if QxM is RF or baseband
        if module.is_rf_type:
            available_slots[f"module{module.slot_idx}"] = ["QCM-RF", "QRM-RF"][
                module.is_qrm_type
            ]
        else:
            available_slots[f"module{module.slot_idx}"] = ["QCM", "QRM"][
                module.is_qrm_type
            ]

# List of all QxM modules present
connect_qxm = widgets.Dropdown(options=[key for key in available_slots.keys()])
display(connect_qxm)

Dropdown(options=('module1', 'module2', 'module3', 'module4', 'module5', 'module6', 'module7', 'module8', 'mod…

In [236]:
# Connect to the cluster QxM module
module = connect_qxm.value
print(module)
qxm = getattr(cluster, module)
print(f"{available_slots[connect_qxm.value]} connected")
print(cluster.get_system_state())

module13
QCM-RF connected
Status: OKAY, Flags: NONE, Slot flags: NONE


In [238]:
qxm.out1_att(30)

## Select complex output

In [239]:
outputs = set(map(lambda i: i.split("_")[0], filter(lambda k: "out" in k and (not k.startswith("_")), dir(qxm))))
select_out = widgets.Dropdown(options = [key for key in outputs])
display(select_out)

Dropdown(options=('out0', 'disconnect', 'out1'), value='out0')

In [240]:
qubits = ['q06','q07','q08','q09','q10','q11','q12','q13','q14','q15']
lo_list = [4741024916,
5162595472,
4485872796,
5417232910,
4602133616,
5085094684,
4658426834,
5306017728,
4644250149,
5051224449]
qubit_lo = dict(zip(qubits, lo_list))
qubit_lo

{'q06': 4741024916,
 'q07': 5162595472,
 'q08': 4485872796,
 'q09': 5417232910,
 'q10': 4602133616,
 'q11': 5085094684,
 'q12': 4658426834,
 'q13': 5306017728,
 'q14': 4644250149,
 'q15': 5051224449}

In [282]:
# lo_frequency = float(input(f"Enter desired LO frequency (Hz) for '{select_out.value}': "))
#lo_frequency = qubit_lo['q06']
lo_frequency = lo_list[8]
lo_frequency = 3.6e9
if qxm.module_type.name == "QRM":
    # qrms only have one complex output
    qxm.out0_in0_lo_en(True)
    qxm.out0_in0_lo_freq(lo_frequency)
else:
    lo_en = getattr(qxm, f"{select_out.value}_lo_en")
    lo_freq = getattr(qxm, f"{select_out.value}_lo_freq")
    lo_en(True)
    lo_freq(lo_frequency)
print(lo_frequency)

3600000000.0


## Select Sequencer in Module

In [275]:
qxm.stop_sequencer()
sequencers = { k : getattr(qxm, k) for k in filter(lambda a: a.startswith("sequencer") and (a != "sequencers"), dir(qxm)) }
select_seq = widgets.Dropdown(options = [key for key in sequencers.keys()])
display(select_seq)

Dropdown(options=('sequencer0', 'sequencer1', 'sequencer2', 'sequencer3', 'sequencer4', 'sequencer5'), value='…

## Upload program to sequencer

In [283]:
seq = sequencers[select_seq.value]
# cz = 500
# MY_IF = (500-cz)*1e6
MY_IF = 250e6
seq.nco_freq(MY_IF)
# seq.nco_freq(MY_IF := float(input("ENTER IF: ")))   # using 10 MHz IF

In [284]:
seq.sequence(os.path.join(os.getcwd(), "sequence.json"))
seq.mod_en_awg(True)
print(MY_IF)
print(seq.name)

250000000.0
clusterA_module13_sequencer0


## Control sliders

In [285]:
rich.print(f"1.\tConnect complex output '{select_out.value}' of cluster '{device_name}' module '{module}' to a spectrum analyser.")
rich.print(f"2.\tIn the spectrum analyser, set the center frequency to {lo_frequency/1e9} GHz")
rich.print(f"3.\tAdjust the gain ratio and phase offset sliders until the LSB is gone.")
rich.print(f"4.\tAdjust the I and Q offsets until the LO peak dissapears.")
rich.print(f"5.\tWhen only RSB is visible in the spectrum analyser, save the calibration with the last cell.")

calibrated_values = [0,0,0.5,0] if seq.seq_idx == 0 else calibration.loc[len(calibration)-1][-4:].values.tolist() # storage array
#calibrated_values = [-0.004462799999999999*1000, -0.0097599*1000, 0.9824, -5.3845]

def set_offset_I(offset_I):
    if qxm.module_type.name == "QRM":
        qxm.out0_offset_path0(offset_I) # qrm have only one output
    else:
        #print(f"{select_out.value}_offset_path0")
        offset_path0 = getattr(qxm, f"{select_out.value}_offset_path0")
        offset_path0(offset_I)

    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[0] = offset_I

def set_offset_Q(offset_Q):
    if qxm.module_type.name == "QRM":
        qxm.out0_offset_path1(offset_Q) # qrm have only one output
    else:
        offset_path1 = getattr(qxm, f"{select_out.value}_offset_path1")
        offset_path1(offset_Q)
        
    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[1] = offset_Q

def set_gain_ratio(gain_ratio):
    seq.mixer_corr_gain_ratio(gain_ratio)
    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[2] = gain_ratio

def set_phase_offset(phase_offset):
    seq.mixer_corr_phase_offset_degree(phase_offset)
    qxm.arm_sequencer(seq.seq_idx)
    qxm.start_sequencer(seq.seq_idx)
    calibrated_values[3] = phase_offset

interact(
    set_offset_I, offset_I=widgets.FloatSlider(min=2, max=3, step=0.00001, value=calibrated_values[0], readout_format='.4f', layout=widgets.Layout(width='95%' ))
)
interact(
    set_offset_Q, offset_Q=widgets.FloatSlider(min=-11, max=-8, step=0.00001, value=calibrated_values[1], readout_format='.4f', layout=widgets.Layout(width='95%' ))
)
interact(
    set_gain_ratio,
    gain_ratio=widgets.FloatSlider(min=0.94, max=0.98, step=0.0001, value=calibrated_values[2], readout_format='.4f', layout=widgets.Layout(width='95%' )),
)
interact(
    set_phase_offset,
    phase_offset=widgets.FloatSlider(min=-45, max=-30, step=0.00001, value=calibrated_values[3], readout_format='.4f', layout=widgets.Layout(width='95%' )),
)

1.      Connect complex output 'out1' of cluster 'clusterA' module 'module13' to a spectrum analyser.

2.      In the spectrum analyser, set the center frequency to 3.6 GHz

3.      Adjust the gain ratio and phase offset sliders until the LSB is gone.

4.      Adjust the I and Q offsets until the LO peak dissapears.

5.      When only RSB is visible in the spectrum analyser, save the calibration with the last cell.

interactive(children=(FloatSlider(value=2.0, description='offset_I', layout=Layout(width='95%'), max=3.0, min=…

interactive(children=(FloatSlider(value=-8.0, description='offset_Q', layout=Layout(width='95%'), max=-8.0, mi…

interactive(children=(FloatSlider(value=0.94, description='gain_ratio', layout=Layout(width='95%'), max=0.98, …

interactive(children=(FloatSlider(value=-30.0, description='phase_offset', layout=Layout(width='95%'), max=-30…

<function __main__.set_phase_offset(phase_offset)>

In [286]:
calibration.loc[len(calibration)] = [module, select_out.value, seq.seq_idx, lo_frequency, MY_IF, *tuple(calibrated_values)]
calibration

,module,complex_output,sequencer index,lo_freq (Hz),if (Hz),dc_mixer_offset_I,dc_mixer_offset_Q,mixer_amp_ratio,mixer_phase_error_deg
0,module8,out0,0,4.900000e+09,-100000000.0,0.00000,0.00000,0.9500,-10.00000
1,module9,out0,0,4.644250e+09,-100000000.0,0.00000,0.00000,0.9500,-10.00000
2,module9,out0,0,4.644250e+09,-100000000.0,-10.39520,-10.04559,0.9665,-16.84362
3,module13,out1,0,3.600000e+09,250000000.0,2.64467,-9.27490,0.9426,-35.30410
4,module13,out1,0,3.400000e+09,450000000.0,2.00000,-8.00000,0.9400,-30.00000
5,module13,out1,0,3.600000e+09,250000000.0,2.71202,-9.12576,0.9413,-35.35221


In [287]:
calibration.to_csv("mixercorrectionLokeA25QFL_q14-q15.csv")

In [152]:
cluster.reset()